In [1]:
#### This notebook to debug the compacting row sample issue, 
## when you drop the 1D feature nan pixel out of the AOI and unstacking again, it will gives you different shape of pixel
## Lesson learn, never drop the pixel, but mask only them, pixel is not vector polygon,
## all index should be the same, and the shape should be the same, if not, you have to debug why the shape is different

## start again, for tsfresh only fix, for the LC_Planet we can generate directly the feature, 
## however, tsfresh feature, we should avoid to redo the long process of ts-fresh extraction for some hours we extrace

In [2]:
# ============================================================================
# STEP 1: ENVIRONMENT SETUP
# ============================================================================
# Load environment variables and initialize Google Earth Engine

from dotenv import load_dotenv
load_dotenv()

import ee, eemont
from forestry_carbon_arr.core import ForestryCarbonARR
from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr

import gcsfs
import os

fs = gcsfs.GCSFileSystem(project=os.getenv("GOOGLE_CLOUD_PROJECT"), token='/usr/src/app/user_id.json')

forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
forestry.initialize_gee()

✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


In [3]:
zarr_filtered_path = f"gs://remote_sensing_saas/01-korindo/timeseries_zarr/ds_filtered_aoi_korindo_buffer.zarr"
ds_filtered_aoi = load_dataset_zarr(zarr_filtered_path, storage='gcs')
ds_filtered_aoi

📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/timeseries_zarr/ds_filtered_aoi_korindo_buffer.zarr
✅ Dataset loaded: {'time': 11, 'x': 4489, 'y': 3213}


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:855: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


<xarray.Dataset> Size: 4GB
Dimensions:  (time: 11, x: 4489, y: 3213)
Coordinates:
  * time     (time) datetime64[ns] 88B 2015-12-31 2016-12-31 ... 2025-12-31
  * x        (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y        (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
Data variables:
    EVI      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    FCD      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    GNDVI    (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    NBR      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    NDVI     (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    SAVI     (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
Attributes:
    crs:      EPSG:32749

In [4]:
print(ds_filtered_aoi)

<xarray.Dataset> Size: 4GB
Dimensions:  (time: 11, x: 4489, y: 3213)
Coordinates:
  * time     (time) datetime64[ns] 88B 2015-12-31 2016-12-31 ... 2025-12-31
  * x        (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y        (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
Data variables:
    EVI      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    FCD      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    GNDVI    (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    NBR      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    NDVI     (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    SAVI     (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
Attributes:
    crs:      EPSG:32749


In [5]:
zarr_path = 'gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/ds_features_all_aoi.zarr'
ds_ml_aoi_tsfresh = load_dataset_zarr(zarr_path)
print(ds_ml_aoi_tsfresh)

📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/ds_features_all_aoi.zarr
✅ Dataset loaded: {'sample': 4073658, 'features': 197}
<xarray.Dataset> Size: 7GB
Dimensions:     (sample: 4073658, features: 197)
Coordinates:
  * features    (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__coe...
  * sample      (sample) int64 33MB 0 1 2 3 ... 4073654 4073655 4073656 4073657
Data variables:
    coord_x     (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    coord_y     (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    plot_id     (sample) object 33MB dask.array<chunksize=(509208,), meta=np.ndarray>
    X_features  (sample, features) float64 6GB dask.array<chunksize=(100000, 197), meta=np.ndarray>


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:855: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


In [6]:
import xarray as xr
import numpy as np
from rasterio.transform import Affine

def unstack_from_ds_sample(ds, var="X_features", crs=None, transform=None, nodata=-9999):
    """Unstack a sample-based dataset back to y/x grid and return a Dataset."""
    da = ds[var].assign_coords(
        x=("sample", ds.coord_x.values),
        y=("sample", ds.coord_y.values),
    )

    da_grid = (
        da
        .set_index(sample=("y", "x"))
        .unstack("sample")
        .rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
    )

    if crs is not None:
        da_grid = da_grid.rio.write_crs(crs, inplace=False)
    if isinstance(transform, Affine):
        da_grid = da_grid.rio.write_transform(transform, inplace=False)

    return da_grid.to_dataset(name=var)

# Example usage: provide real Affine if available
grid_ds = unstack_from_ds_sample(
    ds_ml_aoi_tsfresh,
    var="X_features",
    crs=ds_ml_aoi_tsfresh.attrs.get("crs"),
    transform=None,
)
print(grid_ds)

<xarray.Dataset> Size: 21GB
Dimensions:     (y: 2907, x: 4489, features: 197)
Coordinates:
  * y           (y) float64 23kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
  * x           (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * features    (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__coe...
Data variables:
    X_features  (features, y, x) float64 21GB dask.array<chunksize=(197, 22, 4489), meta=np.ndarray>


In [7]:
## restore, to re-align
import pandas as pd
import xarray as xr

ds_ml_aoi = ds_ml_aoi_tsfresh

In [8]:
import numpy as np, pandas as pd, xarray as xr, gc, dask

dask.config.set({"temporary-directory": "/mnt/data/tmp"})

# 1) stack source to get pixel index (lazy)
band_data = ds_filtered_aoi["EVI"].stack(pixel=("y","x"), create_index=False)
drag_index = band_data.pixel.to_index()
n_pixels_total = band_data.sizes["pixel"]

# 2) compute valid_mask in chunks (same as before)
valid_mask = np.zeros(n_pixels_total, dtype=bool)
chunk_size = 400_000  # smaller to reduce RAM
for i in range(0, n_pixels_total, chunk_size):
    end = min(i + chunk_size, n_pixels_total)
    chunk = band_data.isel(pixel=slice(i, end))
    valid_mask[i:end] = ~np.isnan(chunk.compute().values).all(axis=0)
    del chunk
    gc.collect()
print("✅ valid mask computed")
valid_index = drag_index[valid_mask]
valid_pos = np.nonzero(valid_mask)[0]  # positional indices to avoid misalignment

✅ valid mask computed


In [9]:
ds_ml_aoi

<xarray.Dataset> Size: 7GB
Dimensions:     (sample: 4073658, features: 197)
Coordinates:
  * features    (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__coe...
  * sample      (sample) int64 33MB 0 1 2 3 ... 4073654 4073655 4073656 4073657
Data variables:
    coord_x     (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    coord_y     (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    plot_id     (sample) object 33MB dask.array<chunksize=(509208,), meta=np.ndarray>
    X_features  (sample, features) float64 6GB dask.array<chunksize=(100000, 197), meta=np.ndarray>

In [ ]:
import dask.array as da
import dask, gc, zarr

# 3) create flattened output zarr (pixel, features), no unstack
pix_chunk = 1000_000
feat_chunk = -1
dst_path = "/mnt/data/tmp/X_features_flat.zarr"
n_pix = len(drag_index)
n_feat = ds_ml_aoi.sizes["features"]

data = da.empty((n_pix, n_feat), chunks=(pix_chunk, n_feat), dtype="float32")

# after creating `data` with da.empty(...)
template = xr.DataArray(
    data,
    coords={"pixel": ("pixel", drag_index), "features": ds_ml_aoi.features},
    dims=("pixel", "features"),
    name="X_features",
)
template.to_dataset().to_zarr(
    dst_path,
    mode="w",
    compute=False,      # still lazy init
    zarr_format=2,     # avoid the v3 codec check
    encoding={"X_features": {"compressor": None}},
)

# # initialize store without materializing everything
# ds_template = template.isel(pixel=slice(0, 1)).to_dataset()
# save_dataset_efficient_zarr(ds_template, dst_path,
#                             overwrite=True, chunk_sizes={"pixel": pix_chunk, "features": feat_chunk},
#                             gee_compatible=False)

Delayed('_finalize_store-d306c978-2339-44af-b085-2abd76c4462c')

In [12]:
# !pip install ipywidgets

In [13]:
# write valid rows in batches: map coord_x/coord_y to pixel positions, compute blocks in small parallel groups
# tqdm widgets can fail in this environment; using simple text logging instead

x_vals = ds_filtered_aoi.x.values  # ascending
x_len = len(x_vals)
y_vals = ds_filtered_aoi.y.values  # descending
y_len = len(y_vals)
as_y_asc = y_vals[::-1]

# helper to map coords to flat pixel positions
def coords_to_pos(x_arr, y_arr):
    col_idx = np.searchsorted(x_vals, x_arr)
    row_idx_asc = np.searchsorted(as_y_asc, y_arr)
    row_idx = y_len - 1 - row_idx_asc
    return row_idx * x_len + col_idx

# precompute all destination rows once
all_x = ds_ml_aoi.coord_x.values
all_y = ds_ml_aoi.coord_y.values
all_dest_rows = coords_to_pos(all_x, all_y).astype(np.int64)

arr = zarr.open(dst_path + "/X_features", mode="r+")
# align dask chunk to batch to reduce rechunk overhead
batch = 1000_000
n_parallel = 4  # number of blocks to compute in parallel
pix_chunk = batch
feat_chunk = -1

da = ds_ml_aoi["X_features"].chunk({"sample": pix_chunk, "features": feat_chunk})

total_batches = int(np.ceil(ds_ml_aoi.sizes["sample"] / batch))
batch_idx = 0

for start in range(0, ds_ml_aoi.sizes["sample"], batch * n_parallel):
    tasks = []
    spans = []
    for k in range(n_parallel):
        i = start + k * batch
        if i >= ds_ml_aoi.sizes["sample"]:
            break
        j = min(i + batch, ds_ml_aoi.sizes["sample"])
        spans.append((i, j))
        tasks.append(da.isel(sample=slice(i, j)).astype("float32"))

    blocks = dask.compute(*tasks, scheduler="threads")

    for (i, j), block in zip(spans, blocks):
        block_np = np.ascontiguousarray(block.values)
        dest_rows = all_dest_rows[i:j]
        arr.set_orthogonal_selection((dest_rows, slice(None)), block_np)
        batch_idx += 1
        if batch_idx % 5 == 0 or batch_idx == total_batches:
            print(f"Written {batch_idx}/{total_batches} batches")
    gc.collect()


Written 5/9 batches
Written 9/9 batches


In [14]:
# rebuild MultiIndex on pixel, then unstack safely
import pandas as pd

drag_index = pd.MultiIndex.from_product([ds_filtered_aoi.y.values, ds_filtered_aoi.x.values],
                                        names=["y", "x"])

flat = xr.open_zarr(dst_path)["X_features"].chunk({"pixel": 100_000, "features": -1})
flat = flat.assign_coords(pixel=("pixel", drag_index))

grid = flat.unstack("pixel")
# if unstack already gives y/x, nothing to rename; otherwise handle pixel_level_* names
if "pixel_level_0" in grid.dims and "pixel_level_1" in grid.dims:
    grid = grid.rename({"pixel_level_0": "y", "pixel_level_1": "x"})
elif "y" in grid.dims and "x" in grid.dims:
    pass
else:
    raise ValueError(f"Unexpected dims after unstack: {grid.dims}")

# ensure north-up orientation; unstack sorts y ascending by default
if grid.y[0] < grid.y[-1]:
    grid = grid.sortby("y", ascending=False)

grid = grid.assign_coords(y=ds_filtered_aoi.y, x=ds_filtered_aoi.x)
grid_ds = (
    grid
    .rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
    .rio.write_crs(ds_filtered_aoi.rio.crs, inplace=False)
    .rio.write_transform(ds_filtered_aoi.rio.transform(), inplace=False)
)

In [15]:
grid_ds

<xarray.DataArray 'X_features' (features: 197, y: 3213, x: 4489)> Size: 11GB
dask.array<getitem, shape=(197, 3213, 4489), dtype=float32, chunksize=(197, 19, 4489), chunktype=numpy.ndarray>
Coordinates:
  * features     (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__co...
  * y            (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
  * x            (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
    spatial_ref  int64 8B 0

In [16]:
grid_ds.isel(features=0)

<xarray.DataArray 'X_features' (y: 3213, x: 4489)> Size: 58MB
dask.array<getitem, shape=(3213, 4489), dtype=float32, chunksize=(19, 4489), chunktype=numpy.ndarray>
Coordinates:
    features     <U68 272B 'EVI__fft_coefficient__attr_"angle"__coeff_5'
  * y            (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
  * x            (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
    spatial_ref  int64 8B 0

In [17]:
import rasterio
import numpy as np
import pandas as pd

# Export multiband GeoTIFF (numeric bands only) to GCS


def convert_to_geotiff(ds, band_list, gcs_out, nodata_val=-9999, dtype="int16", 
    extra_attrs=None, class_name_map=None):
    """Export selected bands to GeoTIFF and upload to GCS. Keeps your comments intact."""
    extra_attrs = extra_attrs or {}

    bands = {}
    # bands = {
    #     "el_tsfresh": final_ds.el_tsfresh,
    #     "lc_class": final_ds.lc_class,
    #     "el_lc": final_ds.el_lc,
    #     "eligibility_arr": final_ds.eligibility_arr,
    #     "subtype_zone_code": final_ds.subtype_zone,
    # }
    for name in band_list:
        if name == "subtype_zone_code" and class_name_map:  # encode strings to codes
            name_to_code = {v: k for k, v in class_name_map.items()}
            bands[name] = xr.apply_ufunc(
                lambda v: nodata_val if pd.isna(v) else name_to_code.get(str(v), nodata_val),
                ds.subtype_zone,
                vectorize=True,
                dask="parallelized",
                output_dtypes=[np.int16],
            )
        else:
            bands[name] = ds[name]

    bands_clean = {
        k: v.fillna(nodata_val).astype(dtype).rio.write_nodata(nodata_val)
        for k, v in bands.items()
    }

    stack = xr.concat(
        [bands_clean[k] for k in bands_clean],
        dim="band",
    ).assign_coords(band=list(bands_clean.keys()))

    # carry nodata in metadata for clarity
    stack = stack.assign_attrs(nodata=nodata_val, **extra_attrs)
    stack = stack.rio.write_crs(ds.rio.crs, inplace=False)
    local_out = "/tmp/final_ds.tif"
    stack.rio.to_raster(local_out, driver="GTiff", compress="DEFLATE")

    with rasterio.open(local_out, "r+") as dst:
        dst.descriptions = list(bands_clean.keys())  # ['el_tsfresh', 'lc_class', ...]

    # Upload to GCS
    fs.put(local_out, gcs_out)
    print("uploaded to", gcs_out)


# convert_to_geotiff(
#     final_ds,
#     ["el_tsfresh", "lc_class", "el_lc", "eligibility_arr", "subtype_zone_code"],
#     gcs_out="gs://remote_sensing_saas/01-korindo/final_zone/final_ds.tif",
#     extra_attrs={"subtype_zone_map": class_name_map},
# )


In [18]:
gr = grid_ds.isel(features=0).to_dataset()
gr

<xarray.Dataset> Size: 58MB
Dimensions:      (y: 3213, x: 4489)
Coordinates:
    features     <U68 272B 'EVI__fft_coefficient__attr_"angle"__coeff_5'
  * y            (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
  * x            (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
    spatial_ref  int64 8B 0
Data variables:
    X_features   (y, x) float32 58MB dask.array<chunksize=(19, 4489), meta=np.ndarray>

In [19]:
ds_filtered_aoi

<xarray.Dataset> Size: 4GB
Dimensions:  (time: 11, x: 4489, y: 3213)
Coordinates:
  * time     (time) datetime64[ns] 88B 2015-12-31 2016-12-31 ... 2025-12-31
  * x        (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y        (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
Data variables:
    EVI      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    FCD      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    GNDVI    (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    NBR      (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    NDVI     (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
    SAVI     (time, x, y) float32 635MB dask.array<chunksize=(11, 128, 128), meta=np.ndarray>
Attributes:
    crs:      EPSG:32749

In [20]:
convert_to_geotiff(
    gr.rio.write_crs("EPSG:32749", inplace=False),
    ["X_features"],
    gcs_out="gs://remote_sensing_saas/01-korindo/final_zone/test_AGAIN5.tif",
    # extra_attrs={"subtype_zone_map": class_name_map},
)

uploaded to gs://remote_sensing_saas/01-korindo/final_zone/test_AGAIN5.tif


In [21]:
## good, above tif is align, its shape, let see the content of pixel if they are scrambled or fine
tsfresh_pred_result = forestry.config['tsfresh_pred_result']
lc_pred_result = forestry.config['lc_pred_result']

ds_tsfresh = load_dataset_zarr(tsfresh_pred_result)
ds_lc = load_dataset_zarr(lc_pred_result)


📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/ds_pred_results_tsfresh_aoi.zarr


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:855: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


✅ Dataset loaded: {'sample': 4073658, 'features': 197}
📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/planet_obia_arcgis/final_ds_pred_LC_20251207.zarr
✅ Dataset loaded: {'sample': 4197230, 'feature': 30}


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:855: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


In [22]:
ds_tsfresh

<xarray.Dataset> Size: 7GB
Dimensions:            (sample: 4073658, features: 197)
Coordinates:
  * features           (features) <U68 54kB 'EVI__fft_coefficient__attr_"angl...
  * sample             (sample) int64 33MB 0 1 2 3 ... 4073655 4073656 4073657
Data variables:
    coord_x            (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    coord_y            (sample) float64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    eligibility_clean  (sample) int64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>
    plot_id            (sample) object 33MB dask.array<chunksize=(509208,), meta=np.ndarray>
    X_features         (sample, features) float64 6GB dask.array<chunksize=(100000, 197), meta=np.ndarray>
    y_pred             (sample) int64 33MB dask.array<chunksize=(100000,), meta=np.ndarray>

In [23]:
# ## merging the ds but only extract the important columns (y_pred, pred_class)
import xarray as xr
# do first for el - tsfresh

tsfresh_da= ds_tsfresh.X_features

ds_tsfresh_unstack = (
    tsfresh_da.assign_coords(
        coord_x=("sample", ds_tsfresh.coord_x.data),
        coord_y=("sample", ds_tsfresh.coord_y.data),
    )
    .set_index(sample=("coord_y", "coord_x"))  # build MultiIndex on existing sample
    .unstack("sample")                          # becomes dims coord_y, coord_x
    .rename(coord_y="y", coord_x="x")
    .rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)
    .rio.write_crs('EPSG:32749', inplace=False)
    .to_dataset(name="X_features")
)
ds_tsfresh_unstack

<xarray.Dataset> Size: 21GB
Dimensions:      (y: 2907, x: 4489, features: 197)
Coordinates:
  * y            (y) float64 23kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
  * x            (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * features     (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__co...
    spatial_ref  int64 8B 0
Data variables:
    X_features   (features, y, x) float64 21GB dask.array<chunksize=(197, 22, 4489), meta=np.ndarray>

In [24]:
ds_tsfresh_unstack.isel(features=0)

<xarray.Dataset> Size: 104MB
Dimensions:      (y: 2907, x: 4489)
Coordinates:
  * y            (y) float64 23kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
  * x            (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
    features     <U68 272B 'EVI__fft_coefficient__attr_"angle"__coeff_5'
    spatial_ref  int64 8B 0
Data variables:
    X_features   (y, x) float64 104MB dask.array<chunksize=(22, 4489), meta=np.ndarray>

In [25]:
convert_to_geotiff(
    ds_tsfresh_unstack.isel(features=0).rio.write_crs("EPSG:32749", inplace=False),
    ["X_features"],
    gcs_out="gs://remote_sensing_saas/01-korindo/final_zone/check_ref_value_pix_wrong_shape.tif",
    # extra_attrs={"subtype_zone_map": class_name_map},
)

uploaded to gs://remote_sensing_saas/01-korindo/final_zone/test_AGAIN6.tif


In [27]:
### NOW AFTER CONFIRM IT FIXED!
ds_fixed = grid_ds.to_dataset()
ds_fixed

<xarray.Dataset> Size: 11GB
Dimensions:      (features: 197, y: 3213, x: 4489)
Coordinates:
  * features     (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__co...
  * y            (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
  * x            (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
    spatial_ref  int64 8B 0
Data variables:
    X_features   (features, y, x) float32 11GB dask.array<chunksize=(197, 19, 4489), meta=np.ndarray>

In [34]:
# add constant plot_id across y/x grid
plot_ids = np.full((ds_fixed.sizes['y'], ds_fixed.sizes['x']), 'aoi_1', dtype=object)
ds_fixed['plot_id'] = xr.DataArray(plot_ids, coords={'y': ds_fixed.y, 'x': ds_fixed.x}, dims=('y','x'))
ds_fixed

<xarray.Dataset> Size: 11GB
Dimensions:      (features: 197, y: 3213, x: 4489)
Coordinates:
  * features     (features) <U68 54kB 'EVI__fft_coefficient__attr_"angle"__co...
  * y            (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
  * x            (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
    spatial_ref  int64 8B 0
Data variables:
    X_features   (features, y, x) float32 11GB dask.array<chunksize=(197, 19, 4489), meta=np.ndarray>
    plot_id      (y, x) object 115MB 'aoi_1' 'aoi_1' 'aoi_1' ... 'aoi_1' 'aoi_1'

In [37]:
zarr_path = 'gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/FIXED_ds_pred_results_tsfresh_aoi.zarr'
chunk_sizes = {'features': -1, 'y': 512, 'x': 512}
save_dataset_efficient_zarr(ds_fixed, zarr_path, overwrite=True, gee_compatible=False, 
chunk_sizes=chunk_sizes)


📦 Saving to zarr: gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/FIXED_ds_pred_results_tsfresh_aoi.zarr
   Dimensions: {'features': 197, 'y': 3213, 'x': 4489}
   Chunks: {'features': -1, 'y': 512, 'x': 512}
   Compression: None
   Zarr version: 2
   Consolidated metadata: True
   Storage: gcs


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:641: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"   Dimensions: {dict(ds.dims)}")


💾 Writing to zarr (with automatic parallelism)...


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:729: FutureWarning: zarr_version is deprecated, use zarr_format
  ds_chunked.to_zarr(


[                                        ] | 0% Completed | 115.48 ms

/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:729: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  ds_chunked.to_zarr(


[########################################] | 100% Completed | 229.16 ms
[########################################] | 100% Completed | 73.62 s
✅ Dataset saved successfully!
   Store size: 7.31 GB
   Time: 83.1 seconds (1.4 minutes)
   Write speed: 90.1 MB/s
   Path: gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/FIXED_ds_pred_results_tsfresh_aoi.zarr


'gs://remote_sensing_saas/01-korindo/tsfresh_model_catboost/FIXED_ds_pred_results_tsfresh_aoi.zarr'

In [39]:
### NOW FIXING THE PLANET LC DATA X_features
zarr_path = 'gs://remote_sensing_saas/01-korindo/planet_obia_arcgis/ds_clean_planet_obia_arcgis_202409_202507_median.zarr'
# zarr_path = './01_output/ds_clean_planet_obia_arcgis_202409_202507_median.zarr'

ds_sample = load_dataset_zarr(zarr_path)
ds_sample

📂 Loading dataset from GCS zarr: gs://remote_sensing_saas/01-korindo/planet_obia_arcgis/ds_clean_planet_obia_arcgis_202409_202507_median.zarr


✅ Dataset loaded: {'y': 3224, 'x': 4497}


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:855: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"✅ Dataset loaded: {dict(ds.dims)}")


<xarray.Dataset> Size: 10GB
Dimensions:             (y: 3224, x: 4497)
Coordinates:
  * x                   (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05
  * y                   (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06
Data variables: (12/166)
    ARVI_MAXIMUM        (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    ARVI_MEAN           (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    ARVI_MEDIAN         (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    ARVI_MINIMUM        (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    ARVI_STD            (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    AVI_MAXIMUM         (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    ...                  ...
    VIG_STD             (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    WDRVI_MAXIMUM       (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    WDRVI_MEAN          (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    WDRVI_MEDIAN        (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    WDRVI_MINIMUM       (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
    WDRVI_STD           (y, x) float32 58MB dask.array<chunksize=(256, 256), meta=np.ndarray>
Attributes:
    TIFFTAG_SOFTWARE:          GDAL 3.11.3e (3110322) (4.7.1;1745;3.0.3;2.2.4...
    AREA_OR_POINT:             Area
    RepresentationType:        ATHEMATIC
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        100
    STATISTICS_MEAN:           13.792248932948
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         22.758441987713
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0
    crs:                       EPSG:32749

In [40]:
import numpy as np
import xarray as xr

# Build a 3D X_features (y, x, feature) directly from ds_sample
feature_cols = [f for f in ds_sample.data_vars.keys() if f not in ["target", "plot_id", "X_features"]]
X_features_da = (
    ds_sample[feature_cols]
    .to_array(dim="feature")        # dims: (feature, y, x)
    .transpose("y", "x", "feature") # reorder to (y, x, feature)
    .chunk({"y": 512, "x": 512, "feature": -1})
)

# Attach back to the dataset
ds_aoi_unstack = ds_sample.assign(X_features=X_features_da)
ds_aoi_unstack = ds_aoi_unstack.drop_vars(feature_cols)
ds_aoi_unstack = ds_aoi_unstack.assign_attrs(crs=ds_aoi_unstack.rio.crs)
ds_aoi_unstack
# !pip install geocube

<xarray.Dataset> Size: 10GB
Dimensions:     (y: 3224, x: 4497, feature: 165)
Coordinates:
  * x           (x) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * y           (y) float64 26kB 9.982e+06 9.982e+06 ... 9.949e+06 9.949e+06
  * feature     (feature) object 1kB 'ARVI_MAXIMUM' 'ARVI_MEAN' ... 'WDRVI_STD'
Data variables:
    plot_id     (y, x) object 116MB dask.array<chunksize=(403, 1125), meta=np.ndarray>
    X_features  (y, x, feature) float32 10GB dask.array<chunksize=(512, 512, 165), meta=np.ndarray>
Attributes:
    TIFFTAG_SOFTWARE:          GDAL 3.11.3e (3110322) (4.7.1;1745;3.0.3;2.2.4...
    AREA_OR_POINT:             Area
    RepresentationType:        ATHEMATIC
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        100
    STATISTICS_MEAN:           13.792248932948
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         22.758441987713
    STATISTICS_VALID_PERCENT:  100
    scale_factor:              1.0
    add_offset:                0.0
    crs:                       EPSG:32749

In [42]:
# feature or features, we should fix this later, i prefer with feature, but it seem, we forgot to edit in tsfresh ds
chunk_sizes = {'feature': -1, 'y': 512, 'x': 512}
zarr_path = 'gs://remote_sensing_saas/01-korindo/planet_obia_arcgis/FIX_ds_X_features_planet_obia_arcgis_202409_202507_median.zarr'
save_dataset_efficient_zarr(ds_aoi_unstack, zarr_path, overwrite=True, gee_compatible=False, 
chunk_sizes=chunk_sizes)


📦 Saving to zarr: gs://remote_sensing_saas/01-korindo/planet_obia_arcgis/FIX_ds_X_features_planet_obia_arcgis_202409_202507_median.zarr
   Dimensions: {'y': 3224, 'x': 4497, 'feature': 165}
   Chunks: {'feature': -1, 'y': 512, 'x': 512}
   Compression: None
   Zarr version: 2
   Consolidated metadata: True
   Storage: gcs
💾 Writing to zarr (with automatic parallelism)...


/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:641: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"   Dimensions: {dict(ds.dims)}")
/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:729: FutureWarning: zarr_version is deprecated, use zarr_format
  ds_chunked.to_zarr(


[#                                       ] | 2% Completed | 145.87 ms

/usr/src/app/forestry_carbon_arr/utils/zarr_utils.py:729: SerializationWarning: variable None has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  ds_chunked.to_zarr(


[########################################] | 100% Completed | 2.36 sms
[########################################] | 100% Completed | 437.59 s
✅ Dataset saved successfully!
   Store size: 389.62 MB
   Time: 453.8 seconds (7.6 minutes)
   Write speed: 0.9 MB/s
   Path: gs://remote_sensing_saas/01-korindo/planet_obia_arcgis/FIX_ds_X_features_planet_obia_arcgis_202409_202507_median.zarr


'gs://remote_sensing_saas/01-korindo/planet_obia_arcgis/FIX_ds_X_features_planet_obia_arcgis_202409_202507_median.zarr'